In [ ]:
from propre.rechercheFct import search_Rag
from sentence_transformers import SentenceTransformer,CrossEncoder

embedder = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-electra-base")



In [5]:
query = "Comment isoler ma maison"

top_k = 10
documents = search_Rag(query,embedder,embedder,cross_encoder,cross_encoder,top_k,top_k)


[{'content': "Toute les tuyauteries à températures élevée doivent être isolées. Cependant, appliquée à un réseau de chauffage, l'isolation doit concerner en priorité les parties situées hors volume chauffé car la chaleur est perdue.\n\nLes parties situées en volume chauffé peuvent être moins isolées car la chaleur perdue contribue à chauffer la pièce. De ce fait, le besoin de chauffage est diminué.", 'document': "Isolation d'un réseau hydraulique de chauffage", 'codeK': '291', 'title': '## Application'}, {'content': 'Mettre en place une ventilation par déplacements', 'document': 'Ventilation par déplacements', 'codeK': '1096', 'title': '## Définition'}, {'content': 'La mise en place de vannes d’isolement s’applique aux zones où la production est régulièrement arrêtée, mais elle est aussi primordiale pour isoler les parties de réseaux qui ne sont plus utilisées aujourd’hui à cause d’un déplacement de l’outil de production.\n\nLe zonage réduit les fuites ou les mauvais usages possibles.'

In [ ]:
import requests
from propre.rechercheFct import search_Rag

OLLAMA_API_URL = "http://localhost:11434/api/generate"  # API locale d'Ollama

def query_ollama(question, documents):
    """Envoie une requête à l'API locale d'Ollama pour générer une réponse basée sur des documents fournis."""

    content = [doc["content"] for doc in retour if "content" in doc]  # Récupère tous les documents

    # Vérifier que des documents sont fournis
    if not documents:
        return "Erreur : Aucun document fourni."

    # Concaténer tous les documents en un seul texte
    context = "\n\n".join(content)
    # context = "\n\n".join([f"[Extrait {i+1}]\n{doc}" for i, doc in enumerate(documents)])
    # Construire le prompt pour Gemma
    prompt = f"""DOCUMENTS:\n{context}\n\nQUESTION:\n{question}\n\nINSTRUCTIONS:
    1 . Réponds à la QUESTION en utilisant exclusivement les DOCUMENTS fournis.
    Ne te force pas à utiliser tous les documents
    Ta réponse doit être concise, claire et formulée avec tes propres mots.
    2. Donne des exemples concrets si possible.
    Donne des explications détaillées si nécessaire.
    N'hésite pas à donner plusieurs informations.
    Si la réponse n'est pas contenue dans les documents, réponds simplement que tu n'as pas trouvé d'information sur ce sujet.
    """

    # Préparer le payload pour Ollama
    payload = {
        "model": "gemma:2b",  # Spécifie le modèle utilisé
        "prompt": prompt,
        "stream": False  # False = réponse complète, True = réponse en streaming (optionnel)
    }

    try:
        response = requests.post(OLLAMA_API_URL, json=payload, timeout=60)
        response.raise_for_status()  # Vérifie que la requête est bien passée
        return response.json().get("response", "Erreur : Réponse vide.")
    except requests.exceptions.RequestException as e:
        return f"Erreur lors de la requête Ollama : {str(e)}"

#  Exemple d'utilisation :
question = query
documents = [doc["content"] for doc in retour if "content" in doc]  # Récupère tous les documents

output = query_ollama(question, documents)
print(output)
